TODO

- rewrite the ght_callback 
- think when you apply to filters at the same time
- 

### end tuto and start with the real map

In [2]:
import geopandas as gpd
from bokeh.io import curdoc
from bokeh.plotting import figure, show
from bokeh.models import (GeoJSONDataSource, HoverTool, Div, 
                          Select, RadioGroup, Paragraph, 
                          MultiChoice, CustomJS, ColumnDataSource, DataTable,
                         TableColumn)
from pprint import pprint
from shapely.geometry import Point
import pandas as pd
from bokeh.layouts import row, column

In [3]:
departement = gpd.read_file('data4/bourgogne_fc.shp')

In [4]:
departement.head(n=2)

,code_insee,nom,nuts3,wikipedia,surf_km2,geometry
0,71,Saône-et-Loire,FR263,fr:Saône-et-Loire,8613.0,"POLYGON ((3.62229 46.74091, 3.62230 46.74103, ..."
1,89,Yonne,FR264,fr:Yonne (département),7461.0,"POLYGON ((2.84849 47.72603, 2.84963 47.72627, ..."


In [5]:
departement_source = GeoJSONDataSource(geojson=departement.to_json())

In [6]:
CRS = departement.crs

In [7]:
ght = pd.read_csv('data4/ght.csv', index_col=0)

In [8]:
coord = pd.read_csv('data4/coord_with_cities.csv', index_col=0)

In [9]:
coord['lieu'] = coord.ville.fillna(coord.municipalite)

In [12]:
coordght

,index,etablissement,adresse,latitude,longitude,departement,municipalite,code_postal,village,ville,lieu,nom_ght
0,0,AIHP CENTRE ARMANCON,18 Bis Rue Pierre Semard,47.959810,3.53041,Bourgogne-Franche-Comté,Auxerre,89400,Migennes,NaN,Auxerre,NaN
1,1,C.R.B LES ROSIERS,45 Bd Henri Bazin,47.300410,5.01900,Bourgogne-Franche-Comté,Dijon,21000,NaN,Dijon,Dijon,NaN
2,2,CENTRE COLBERT MEDECINE ET SSR,4 Rue Étienne Litaud,46.994690,3.15970,Bourgogne-Franche-Comté,Nevers,58000,NaN,NaN,Nevers,GHT DE LA NIEVRE
3,3,CENTRE DE CONVALESCENCE,67 Rte d'Ahuy,47.345830,5.03471,Bourgogne-Franche-Comté,Dijon,21121,Fontaine-lès-Dijon,NaN,Dijon,NaN
4,4,CENTRE MEDICAL DE LA VENERIE,Lieu Dit La Vénerie,47.266800,3.28960,Bourgogne-Franche-Comté,NaN,58700,NaN,Saint-Bonnot,Saint-Bonnot,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
87,88,POLYCLINIQUE DU VAL DE LOIRE,49 Bd Jérôme Trésaguet,46.995640,3.14828,Bourgogne-Franche-Comté,NaN,58000,NaN,Nevers,Nevers,NaN
88,89,CLINIQUE PAUL PIQUET,12 Rue Pierre Castets,48.207120,3.29428,Bourgogne-Franche-Comté,NaN,89100,NaN,Sens,Sens,NONE
89,90,CH ROBERT MORLEVAT,3 Av. Pasteur,47.495749,4.34625,Bourgogne-Franche-Comté,Montbard,21140,NaN,NaN,Montbard,GHT COTE D'OR - HAUTE MARNE
90,91,POLYCLINIQUE DU PARC DREVON,18 Cr Général de Gaulle,47.312170,5.04348,Bourgogne-Franche-Comté,NaN,21000,NaN,Dijon,Dijon,NONE


In [10]:
coordght= pd.merge(coord, ght, how='left')

In [34]:
coordght['nom_ght'] = coordght['nom_ght'].fillna('NONE')

In [36]:
coordght.to_csv('data4/coordght.csv')

In [37]:
coordght = pd.read_csv('data4/coordght.csv', index_col=0)

In [10]:
geometry = [Point(xy) for xy in zip(coordght.longitude, coordght.latitude)]
etablissement = gpd.GeoDataFrame(coordght.drop(['longitude', 'latitude'], axis=1), crs=CRS, geometry=geometry)

In [11]:
etablissement_source = GeoJSONDataSource(geojson=etablissement.to_json()) 

In [12]:
'''
source = GeoJSONDataSource(
    geojson=gpd.GeoDataFrame(dict(geometry=[], adresse=[], etablissement=[], index=[])).to_json()
)
'''

source = GeoJSONDataSource(geojson=etablissement.copy().to_json())

In [13]:
p = figure(
    title='New test map',
    x_axis_location=None, 
    y_axis_location=None,
    #sizing_mode='stretch_width',
    width=1200,
    height=750,
    #tooltips=TOOLTIPS
)

In [14]:
patches = p.patches('xs', 
          'ys', 
          source=departement_source,
          fill_color='blue',
          fill_alpha=0.5, 
          line_color="black", 
          line_width=0.5)

In [15]:
circles = p.circle(
    'x',
    'y',
    source=source,
    color='red', 
    size=6,
    hover_color='yellow'
)

In [16]:
TOOLTIPS = [
    ('(x, y)', '($x, $y)'),
    ('etablissement', '@etablissement'),
    ('village', '@village'),
    ('ght', '@nom_ght')
]

In [17]:
ht = HoverTool(
    tooltips= TOOLTIPS,
    renderers = [circles]
)

In [18]:
p.tools.append(ht)

In [19]:
#filter by ght

ght_names = ght.nom_ght.drop_duplicates().sort_values().tolist()
ght_names.append('ALL')
select = Select(title="GHT:", value=ght_names[-1], options=ght_names)

In [20]:
ght_callback = CustomJS(args=dict(static_source=etablissement_source, source= source), code="""
        const static_data = static_source.data;
        var data = source.data;
        const n = static_data.nom_ght.length;
        const widget_value = cb_obj.value;
        
        //reinitilize source data
        for (const key in data){
            data[key] = [];
        }
        if(widget_value == 'ALL') {
            for (let i=0; i < n; i++){
                for (const key in static_data){
                        data[key].push(static_data[key][i])
                    }
                }
        }    
        else{
            //get the data related to the selected etablissement
            for (let i=0; i < n; i++){
                if(static_data.nom_ght[i] == widget_value){
                    for (const key in static_data){
                        data[key].push(static_data[key][i])
                    }
                }
            }          
        }
        source.change.emit();
    """)
select.js_on_change('value', ght_callback)

In [21]:
etab_names = coord.etablissement.drop_duplicates().tolist()

mc = MultiChoice(value=[etab_names[0]], options=etab_names)

In [22]:
flux = pd.read_csv('data4/flux.csv', index_col=0)
static_flux_source = ColumnDataSource(flux)
flux_source = ColumnDataSource(flux.copy())

#etablissement, specialite, provenance, effectif_transfert
columns = [
    TableColumn(field='effectif_transfert', title='Effectif') #formatter=
]
data_table = DataTable(source=flux_source, columns=columns) #width=400, height=280)

In [23]:
etab_callback = CustomJS(args=dict(static_source=etablissement_source, source= source, mc=mc, 
                                   static_flux_source= static_flux_source, 
                                   flux_source=flux_source, etab_names=etab_names), code="""
        const static_data = static_source.data;
        var data = source.data;
        const static_flux_data = static_flux_source.data;
        var flux_data = flux_source.data;
        const n = static_data.nom_ght.length;
        
        //reinitialize source data
        for (const key in data){
            data[key] = [];
        }
        
        //reinitialize flux source data
        for (const key in flux_data){
            flux_data[key] = [];
        }
        
        /*
        //case the is nothing in the multiplechoice
        if (mc.value.lenght == 0){
            mc.value = etab_names
        }
        */
        
        
        //we retrieve the selected etablissement(s) name(s) in the multiplechoice
        //const widget_value = cb_obj.value;
        var widget_value
        
        //loop over all the etablissements into the multiple choices
        for (let j=0; j < mc.value.length; j++){
            widget_value = mc.value[j]
            //get the data related to the selected etablissement
            for (let i=0; i < n; i++){
                if(static_data.etablissement[i] == widget_value){
                    for (const key in static_data){
                        data[key].push(static_data[key][i])
                    }
                }
            }
            //for the flux
            for (let j=0; mc.value.length;j++){
                for(let i=0; i<static_flux_data.etablissement.length;i++){
                    for(const key in static_flux_data){
                        flux_data[key].push(static_flux_data[key][i])
                    }
                }
            }
        }
        
        flux_source.change.emit();
        source.change.emit();
    """)
mc.js_on_change('value', etab_callback)
#select.js_on_change('value', ght_callback)

In [24]:
widgets = column(select, mc, data_table)
layout = row(p, widgets)

In [25]:
show(layout)